Agricultural Land Classification data for England can be downloaded here https://data.gov.uk/dataset/952421ec-da63-4569-817d-4d6399df40a1/provisional-agricultural-land-classification-alc 

Leave the 'Full dataset' toggle checked and choose your favourite format. 

Here we use gdb guessing which one will be best... I have no idea.. 

When you click on the big down arrow, you should get a file called NE_AgriculturalLandClassificationProvisionalEngland_FGDB_Full.zip 

Unzip this, and you'll have a lot of files. The one you need is ?? tbc??!